In [2]:
from ultralytics import YOLO
model = YOLO(r"D:\OneDrive\Bangkit Academy\Project\Image Based\1\runs\segment\train3\weights\best.pt")

image_path = "test1.jpg" 
image_cloth_path = "cloth3.jpg"
results = model.predict(image_path)
boxes = results[0].boxes.xyxy.tolist()
result = results[0]



image 1/1 d:\OneDrive\Bangkit Academy\Project\Image Based\Train\test1.jpg: 640x512 1 Upper-Body, 199.9ms
Speed: 3.0ms preprocess, 199.9ms inference, 234.8ms postprocess per image at shape (1, 3, 640, 512)


In [15]:
print(results[0])

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'Upper-Body'}
obb: None
orig_img: array([[[216, 217, 215],
        [216, 217, 215],
        [216, 217, 215],
        ...,
        [212, 212, 212],
        [212, 212, 212],
        [212, 212, 212]],

       [[216, 217, 215],
        [216, 217, 215],
        [216, 217, 215],
        ...,
        [212, 212, 212],
        [212, 212, 212],
        [212, 212, 212]],

       [[216, 217, 215],
        [216, 217, 215],
        [216, 217, 215],
        ...,
        [212, 212, 212],
        [212, 212, 212],
        [212, 212, 212]],

       ...,

       [[217, 219, 219],
        [217, 219, 219],
        [217, 219, 219],
        ...,
        [209, 211, 211],
        [209, 211, 211],
        [209, 211, 211]],

       [[217, 219, 219],
        [217, 219, 219],
        [217, 219, 219],
        ...,
        [209, 211, 211],

In [133]:
print(boxes[0])

[147.89598083496094, 238.03826904296875, 434.2268981933594, 553.4103393554688]


In [3]:
bounding_height = int(boxes[0][3] - boxes[0][1])
bounding_width = int(boxes[0][2] - boxes[0][0])

print(bounding_height)
print(bounding_width)

431
350


In [4]:
from PIL import Image
masks = result.masks
mask1 = masks[0]
polygon = mask1.xy[0]
mask = mask1.data[0].numpy()
mask_img = Image.fromarray(mask,"I")
mask_img = mask_img.convert('RGB')

In [10]:
print(mask)

[[          0           0           0 ...           0           0           0]
 [          0           0           0 ...           0           0           0]
 [          0           0           0 ...           0           0           0]
 ...
 [          0           0           0 ...           0           0           0]
 [          0           0           0 ...           0           0           0]
 [          0           0           0 ...           0           0           0]]


In [5]:

image = Image.open(image_path)

# Mendapatkan ukuran citra
width, height = image.size
resized_mask_img = mask_img.resize((width, height), Image.ANTIALIAS)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18880\1947126645.py:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_mask_img = mask_img.resize((width, height), Image.ANTIALIAS)


In [6]:
width, height = resized_mask_img.size
resized_mask_img_pixels = resized_mask_img.load()
print("Lebar citra:", width, "pixels")
print("Tinggi citra:", height, "pixels")
#sudah sesuai

Lebar citra: 564 pixels
Tinggi citra: 751 pixels


In [7]:
white_area = (int(boxes[0][0]), int(boxes[0][1]), int(boxes[0][2]), int(boxes[0][3]))
scale_factor = 1.05

# Calculate the expansion size
expand_width = int(((white_area[2] - white_area[0]) * (scale_factor - 1)) / 2)
expand_height = int(((white_area[3] - white_area[1]) * (scale_factor - 1)) / 2)

# Adjust the coordinates
left = (white_area[0] - expand_width)
top = (white_area[1] - expand_height)
right =(white_area[2] + expand_width)
bottom = (white_area[3] + expand_height)

# Crop the adjusted white area
white_part = resized_mask_img.crop(box=(white_area[0], white_area[1], white_area[2], white_area[3]))
white_part_resized = white_part.resize((right-left,bottom-top), Image.ANTIALIAS)

# Create a new blank image that will show changes
new_img = Image.new('RGB', resized_mask_img.size, (0, 0, 0))
new_img.paste(white_part_resized, (left, top))

# Show and save the result
new_img.show()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18880\279353294.py:16: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  white_part_resized = white_part.resize((right-left,bottom-top), Image.ANTIALIAS)


In [139]:
white_part.show()

In [8]:
# Mendapatkan objek pixel dari gambar
pixels = new_img.load()

In [9]:
threshold_value = 128
for y in range(height):
    for x in range(width):
        # Ambil nilai piksel (dalam kasus RGB, ambil salah satu komponen saja karena awalnya adalah grayscale)
        r, g, b = pixels[x, y]  # Semua nilai r, g, b sama karena asalnya grayscale
        if r < threshold_value:
            pixels[x, y] = (0, 0, 0)  # Set pixel to black
        else:
            pixels[x, y] = (255, 255, 255)

In [10]:
image2 = Image.open(image_cloth_path)
#pixels_cloth = image2.load()

In [11]:
adjust_cloth = 150
cloth_resized= image2.resize((bounding_width + adjust_cloth, bounding_height + adjust_cloth), Image.ANTIALIAS)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18880\3427305150.py:2: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  cloth_resized= image2.resize((bounding_width + adjust_cloth, bounding_height + adjust_cloth), Image.ANTIALIAS)


In [12]:
mask_pixels = new_img.load()
width_mask, height_mask = new_img.size

In [13]:
cloth_resized_pixels = cloth_resized.load()
width_cloth,height_cloth = cloth_resized.size

In [14]:
print(cloth_resized_pixels[100,204])

(63, 79, 93)


In [15]:
start_x = int(boxes[0][0]) - int(adjust_cloth / 2)
end_x = int(boxes[0][2]) + int(adjust_cloth / 2)
start_y = int(boxes[0][1]) - int(adjust_cloth / 2)
end_y = int(boxes[0][3]) + int(adjust_cloth / 2)

In [16]:
for y in range(start_y, end_y-1):
    for x in range(start_x, end_x-1):
        current_pixel = mask_pixels[x, y]
        if current_pixel == (0, 0, 0):
            mask_pixels[x, y] = mask_pixels[x,y]  
        else:
            mask_pixels[x, y] = cloth_resized_pixels[x-start_x,y-start_y]

In [17]:
new_img.show()

In [18]:
final_masking = new_img.copy()
final_masking_pixels = final_masking.load()

In [19]:
final_image_ori = Image.open(image_path)
final_image_ori_pixels = final_image_ori.load()

for y in range(height):
    for x in range(width):
        current_pixel_2 = final_masking_pixels[x,y]
        if current_pixel_2 == (0,0,0):
            final_masking_pixels[x,y] = final_image_ori_pixels[x,y]

In [20]:
final_masking.show()